In [10]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier

In [11]:
mp_holistic = mp.solutions.holistic

In [12]:
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not open or find the image: {image_path}")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_holistic.Holistic(static_image_mode=True) as holistic:
        results = holistic.process(image)
        
        if results.pose_landmarks:
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in results.pose_landmarks.landmark]
        else:
            pose_landmarks = [[0, 0, 0]] * 33  # Adjusted number of pose landmarks
        
        if results.left_hand_landmarks:
            left_hand_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in results.left_hand_landmarks.landmark]
        else:
            left_hand_landmarks = [[0, 0, 0]] * 21  # Adjusted number of hand landmarks
        
        if results.right_hand_landmarks:
            right_hand_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in results.right_hand_landmarks.landmark]
        else:
            right_hand_landmarks = [[0, 0, 0]] * 21  # Adjusted number of hand landmarks

        landmarks = pose_landmarks + left_hand_landmarks + right_hand_landmarks
        return np.array(landmarks).flatten()


In [13]:
data_dir = "ASL/asl_alphabet_train"
labels = []
data = []

for class_label, class_dir in enumerate(os.listdir(data_dir)):
    if os.path.isdir(os.path.join(data_dir, class_dir)):
        for filename in os.listdir(os.path.join(data_dir, class_dir)):
            if filename.endswith(".jpg") or filename.endswith(".png"):  # adjust as necessary
                image_path = os.path.join(data_dir, class_dir, filename)
                landmarks = extract_landmarks(image_path)
                data.append(landmarks)
                labels.append(class_label)

KeyboardInterrupt: 

In [14]:
data = np.array(data)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [17]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()



























































# OpenCV Intgration

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
model = load_model("ASL_Media.h5")
class_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [4]:
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = np.expand_dims(frame, axis=0)
    frame = frame / 255.0
    return frame

In [5]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        continue
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(frame_rgb)
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    # Preprocess the frame and make a prediction
    if result.multi_hand_landmarks:
        frame_preprocessed = preprocess_frame(frame_rgb)
        preds = model.predict(frame_preprocessed)
        class_id = np.argmax(preds[0])
        class_label = class_names[class_id]
        
        # Display the prediction on the frame
        cv2.putText(frame, f'Predicted Class: {class_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the frame
    cv2.imshow('ASL Recognition', frame)
    
    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 21ms/step
